# Deep Maxout Network avec entraînement adversariale et évaluation sur base de teste adversariale

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from maxout import CustomMaxout
from adv_attack import create_adv_test
import matplotlib.pyplot as plt
import numpy as np


## 2. Chargement des données

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Normalize the images to [-1, 1]
])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)
batch_size = 64

training_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

## 3. Défintion de la fonction de coût
$$\tilde{J}(\theta, x, y) = \alpha J(\theta, x, y) + (1 - \alpha) J(\theta, x + \epsilon \cdot \text{sign}(\nabla_x J(\theta, x, y)))$$

On définit dans un premier temps une loss de base $$J(\theta, x, y)$$ Comme rien n'est précisé dans l'article, on choisit la cross-entropy.

In [ ]:
# Fonction de perte standard
def loss_fn(model, x, y):
    output = model(x)
    return F.cross_entropy(output, y)

# Fonction de perte adversariale
def adversarial_loss_fn(model, x, y, epsilon, alpha):
    # Calcul de la perte standard
    standard_loss = loss_fn(model, x, y)
    
    # Génération de l'exemple adverse
    x_adv = x + epsilon * torch.sign(torch.autograd.grad(standard_loss, x, create_graph=True)[0])
    
    # Calcul de la perte sur l'exemple adverse
    adversarial_loss = loss_fn(model, x_adv, y)
    
    # Combinaison des deux pertes
    return alpha * standard_loss + (1 - alpha) * adversarial_loss

## 4. Création des modèles utiles

On a besoin d'un modèle  à 240 unit per layer et d'un autre  à 1600

In [ ]:
n_channels = 1
dropout = 0.5

#on créé le premier model qui à 240 unit per layer model
Maxout_240U_Model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28*n_channels, 240),
    CustomMaxout(240, 200, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(200, 160),
    CustomMaxout(160, 120, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(120, 80),
    CustomMaxout(80, 40, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(40, 10),
    nn.LogSoftmax(dim=1)
)

#on créé le second modèle qui a 1600 unit per layer
Maxout_1600U_Model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28*n_channels, 1600),
    CustomMaxout(1600, 1500, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(1500, 1400),
    CustomMaxout(1400, 1300, n_channels, True),
    nn.Linear(1300, 1200),
    CustomMaxout(1200, 1100, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(1100, 1000),
    CustomMaxout(1000, 800, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(800, 600),
    CustomMaxout(600, 400, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(400, 200),  
    CustomMaxout(200, 100, n_channels, True),
    nn.Dropout(dropout),
    nn.Linear(100, 50),
    CustomMaxout(50, 25, n_channels, True),  
    nn.Dropout(dropout),
    nn.Linear(25, 10),  
    nn.LogSoftmax(dim=1)
)

## 5. Entraînement

In [ ]:
# Initialize the early stopping variables
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_valid_loss = float('inf')
epochs_no_improve = 0
early_stop_epochs = 10  
epsilon = 0.1
alpha = 0.5
# Define the number of epochs
n_epochs = 30


# Define the training and validation data loaders
train_dataloader = training_dataloader
valid_dataloader = test_dataloader

model_dict = {"Maxout_240U_Model": Maxout_240U_Model, "Maxout_1600U_Model": Maxout_1600U_Model}


for model_name, model in model_dict.items(): 
   #Train the model
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        correct_train_preds = 0
        total_train_preds = 0
        for batch in train_dataloader:
            inputs, labels = batch
            # Move the inputs and labels to the device
            inputs = inputs.to(device).requires_grad_()
            labels = labels.to(device).requires_grad_()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = adversarial_loss_fn(model, inputs, labels, epsilon, alpha)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train_preds += labels.size(0)
            correct_train_preds += (predicted == labels).sum().item()

        train_losses.append(train_loss / len(train_dataloader))
        train_accuracies.append(100 * correct_train_preds / total_train_preds)
        
    # Plot the training losses and accuracies    
    fig, ax1 = plt.subplots(figsize=(12, 4))

    color = 'tab:red'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Training loss', color=color)
    ax1.plot(train_losses, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Training accuracy', color=color) 
    ax2.plot(train_accuracies, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    plt.show()

    model.eval()
    valid_loss = 0
    correct_valid_preds = 0
    total_valid_preds = 0
    for batch in valid_dataloader:
        inputs, labels = batch
        # Move the inputs and labels to the device
        inputs = inputs.to(device).requires_grad_()
        labels = labels.to(device).requires_grad_()

        outputs = model(inputs)

        loss = adversarial_loss_fn(model, inputs, labels, epsilon, alpha)

        valid_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_valid_preds += labels.size(0)
        correct_valid_preds += (predicted == labels).sum().item()

    valid_losses.append(valid_loss / len(valid_dataloader))
    valid_accuracies.append(100 * correct_valid_preds / total_valid_preds)

    #early stopping
    # Check if the validation loss has improved
    if valid_losses[-1] < best_valid_loss:
        best_valid_loss = valid_losses[-1]
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # If the validation loss hasn't improved for early_stop_epochs, stop training
    if epochs_no_improve == early_stop_epochs:
        print("Early stopping!")
        break

        
    print(f'Validation loss: {valid_losses[-1]:.3f}.. '
          f'Validation accuracy: {valid_accuracies[-1]:.3f}')


    

## 5. Création set de test adversarial

In [ ]:
#on récupère les images et les labels de la base de test
x_test = torch.cat([images for images, labels in test_dataloader]).to(device) #images de la base test
y_test = torch.cat([labels for images, labels in test_dataloader]).to(device) #labels de la base test

eps = 0.1
loss_func = nn.CrossEntropyLoss()

#création des images adverses pour le réseau  à 240 unites
altered_test_240U = create_adv_test(Maxout_240U_Model, x_test, y_test, eps, loss_func)

#création des images adverses pour le réseau  à 1600 unites
altered_test_1600U = create_adv_test(Maxout_1600U_Model, x_test, y_test, eps, loss_func)


## 6 . évaluation des modèles sur la nouvelle base de test adversariale

In [ ]:
total_test_preds = 0
correct_test_preds = 0
test_loss = 0
misclassified_confidences = []
for model_name, model in model_dict.items():
    model.eval()  
    for batch in test_dataloader:
        inputs, labels = batch
        inputs = inputs.to(device).requires_grad_()
        labels = labels.to(device)
    
        # Calculate adversarial loss and generate adversarial examples
        loss = adversarial_loss_fn(model, inputs, labels, epsilon, alpha)
        test_loss += loss.item()
    
        # Disable gradient calculation for prediction and accuracy calculation
        with torch.no_grad():
            outputs = model(inputs)
            probabilities = torch.exp(outputs)  # Convert log probabilities to probabilities
            _, predicted = torch.max(probabilities.data, 1)
    
        total_test_preds += labels.size(0)
        correct_test_preds += (predicted == labels).sum().item()
    
        # Calculate confidence
        confidence = torch.max(probabilities, dim=1)[0]
    
        # Store confidence of misclassified examples
        misclassified = predicted != labels
        misclassified_confidences.extend(confidence[misclassified].tolist())
    
    test_accuracy = 100 * correct_test_preds / total_test_preds
    print(f'Test loss: {test_loss / len(test_dataloader):.3f}.. '
          f'Test accuracy: {test_accuracy:.3f}')
    print(f'Average confidence of misclassified examples: {np.mean(misclassified_confidences):.3f}\n')
    print("===============================================================================")